In [ ]:
!pip install --upgrade xlrd

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Data_BlackKoffer_intern/Penny Pickers_EXPE5/4.xls',sheet_name='results',header=None)
df

In [ ]:
def f1(sigma,t):   # t is the timestamp
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

We win 2dollars on winning bet and loss $40 on lossing bet

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:   # sigma = 0 means that we do not have any data yet to calculate sigma
    return 0.0001  # very little probability of hitting target
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),-4)
  result = 1 - 2*(phi)
  if result>=1: 
    result = 0.999   # near certain of hitting the target
  return result 

In [ ]:
def pc():
  return 0.844

In [ ]:
def compute_expValue(DA,beta,alpha1,alpha2,alpha3,lamda,t):
  if t%20==0 or t%20==1: return 0   # t is the first or the last data point of a trial set
  if t%20<4: DA=0
  Pg = pg(t)
  Pc = pc()
  a = np.exp(-(-math.log(Pg))**alpha3)
  b = lamda*np.exp(-(-math.log(1-Pg))**alpha3)
  c = np.exp(-(-math.log(Pc))**alpha3)
  d = lamda*np.exp(-(-math.log(1-Pc))**alpha3)
  V = Pm1Xt(t-1)*(a*pow(2+DA,alpha1) - b*pow(40,alpha2)) + Pm2Xt(t-1)*(c*pow(2+DA,alpha1) - d*pow(40,alpha2))
  return V # expected value

We bet on trial t+1 if and only if Vt > 0 

In [ ]:
def predict(DA,beta,alpha1,alpha2,alpha3,lamda,starting,ending):
  total = 0
  plist = []  # list of outcomes (bet or not bet)
  expList = []  # list of expected values for each time stamp
  probList = []
  for i in range(starting,ending): 
    # i is the time stamp. 
    Vt = compute_expValue(DA,beta,alpha1,alpha2,alpha3,lamda,i) # Computing expected value for timestamp i
    #print("beta,Vt", beta,Vt)
    h = 1 / (1+math.exp(-beta*Vt)) 
    expList.append(Vt)
    probList.append(h)
    if h>0.5:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: # in the dataframe, index of data at time i+1 is i
        total+=2 
      else: 
        total-=40
    else: plist.append(0)        # adding value for timestamp i+1
  return (plist,expList,probList,total)       

Now we train the model (the parameter DA)

In [ ]:
Y = df[5]

In [ ]:
def errorFunc(prediction, Y):  # cost function
  total = 0
  for i in range(0,160):  # i is timestamp
    #total += -(Y[i]*math.log(prediction[i-1-160]) + (1-Y[i])*math.log(1-prediction[i-1-160]))
    total += -(Y[i]*math.log(prediction[i]) + (1-Y[i])*math.log(1-prediction[i]))
  total /= 160
  return total  

In [ ]:
def derivative_cost_function(DA,beta,alpha1,alpha2,alpha3,lamda):  # gradient of cost function (used in gradient descent)
  G_a1 = 0   # total gradient 
  G_a2 = 0
  G_a3 = 0
  G_L = 0
  G_DA = 0
  G_b = 0
  plist,expList,probList,total = predict(DA,beta,alpha1,alpha2,alpha3,lamda,0,160)
  
  for i in range(1,160):  
    if (i+1)%20<5: continue
    pm1 = Pm1Xt(i-1)
    pm2 = Pm2Xt(i-1)
    Pg = pg(i)
    Pc = pc()
    #V = compute_expValue(i-1,a1,a2,a3,L)
    d_a1 = pm1*(np.exp(-(-math.log(Pg))**alpha3)*(2+DA)**alpha1*math.log(2+DA)) + pm2*(np.exp(-(-math.log(Pc))**alpha3)*(2+DA)**alpha1*math.log(2+DA))          #Derivative(V,a1)
    d_a2 = -pm1*lamda*(np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2) - pm2*lamda*(np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2)
    d_a3 = pm1*(-(-math.log(Pg))**alpha3*math.log(-math.log(Pg))*np.exp(-(-math.log(Pg))**alpha3)*(2+DA)**alpha1 + lamda*(-math.log(1-Pg))**alpha3*math.log(-math.log(1-Pg))*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2) + pm2*(-(-math.log(Pc))**alpha3*math.log(-math.log(Pc))*np.exp(-(-math.log(Pc))**alpha3)*(2+DA)**alpha1 + lamda*(-math.log(1-Pc))**alpha3*math.log(-math.log(1-Pc))*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2)
    d_L = pm1*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2 + pm2*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2
    #d_DA = pm1*(np.exp(-(-math.log(Pg))**alpha3)*alpha1*(2+DA)**(alpha1-1)) + pm2*(np.exp(-(-math.log(Pc))**alpha3)*alpha1*(2+DA)**(alpha1-1))
    d_DA = (pg(i-1)*Pm1Xt(i-2) + pc()*Pm2Xt(i-2))
    
    G_a1 += (probList[i]-Y[i])*beta*d_a1/160
    G_a2 += (probList[i]-Y[i])*beta*d_a2/160
    G_a3 += (probList[i]-Y[i])*beta*d_a3/160
    G_L += (probList[i]-Y[i])*beta*d_L/160
    G_DA += (probList[i]-Y[i])*beta*d_DA
    G_b += (probList[i]-Y[i])*expList[i]
  
  G_a1 /= 160
  G_a2 /= 160
  G_a3 /= 160
  G_L /= 160 
  G_DA /= 160
  G_b /=160
  #print('G_a1,G_a2,G_a3,G_L', G_a1,G_a2,G_a3,G_L)
  return (G_DA,G_b,G_a1,G_a2,G_a3,G_L)

gradient descent

In [ ]:
DA = 0.01
alpha1,alpha2,alpha3,lamda = 1.0,1.0,1.0,1.0
beta = 1
alpha = 0.1
i = 0
while True:
  if DA>=5: break
  
  if i==0:
    pre_DA,pre_beta,pre_a1,pre_a2,pre_a3,pre_L = DA,beta,alpha1,alpha2,alpha3,lamda
    i+=1
  else:
    G_DA,G_b,G_a1,G_a2,G_a3,G_L = derivative_cost_function(DA,beta,alpha1,alpha2,alpha3,lamda)
    DA = DA - alpha*G_DA
    beta = beta - alpha*G_b
    temp1 = alpha1 - alpha*G_a1
    temp2 = alpha2 - alpha*G_a2
    temp3 = alpha3 - alpha*G_a3
    temp4 = lamda - alpha*G_L
    alpha1,alpha2,alpha3,lamda = temp1,temp2,temp3,temp4
    if alpha1>1: alpha1 = 1
    if alpha2>1: alpha2 = 1
    if alpha3>1: alpha3 = 1
    if lamda<=1: lamda = 1.01
    if DA<=0: 
      DA=0.01
      break
    # abs(alpha1-pre_a1)<1e-3 or abs(alpha2-pre_a2)<1e-3 or abs(alpha3-pre_a3)<1e-3 or abs(lamda-pre_L)<1e-3 
    print("DA, beta, alpha1, alpha2, alpha3, lamda: ",DA,beta,alpha1,alpha2,alpha3,lamda)
    if abs(DA-pre_DA)<1e-3 or abs(beta-pre_beta)<1e-3: break
    else: pre_DA,pre_beta,pre_a1,pre_a2,pre_a3,pre_L = DA,beta,alpha1,alpha2,alpha3,lamda
    i+=1
print("DA, beta, alpha1, alpha2, alpha3, lamda: ",DA,beta,alpha1,alpha2,alpha3,lamda)

Tesing the resulting model on remaining 7 sessions and calculating accuracy score

In [ ]:
plist,expList,probList,total = predict(0,beta,alpha1,alpha2,alpha3,lamda,160,300)   # prediction

print(len(plist))
print(total)

In [ ]:
print(plist)

In [ ]:

total = 0
Y = df[5]
print(Y[12]) 
for i in range(0,140):  # we will have to change it to (0,160) when the dataset has a length of 320
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model with CbD is: ",total/140*100,"%")   # similarly here 140 -> 160